In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "baseline"
MODEL_NAME = "baseline"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 4000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 30
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.7MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [6]:
train_df.op_gender.value_counts()

M    7024476
W    2854540
Name: op_gender, dtype: int64

In [8]:
dev_df.op_gender.value_counts()

M    1948371
W     344536
Name: op_gender, dtype: int64

In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [0]:
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [15]:
time_start = time.time()

tokenizer = Tokenizer(num_words=200000)
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 10 minutes, 18 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [18]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   89.8 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
\
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [20]:
# trying to figure out which words are empty here
counter = 0
empty_indexes = []
for index, row in enumerate(embedding_matrix):
  if sum(row) == 0:
    empty_indexes.append(index)
    counter += 1
  if counter > 1000:
    break

for idx in empty_indexes:
  try:
    print(tokenizer.index_word[idx])
  except:
    print("No entry for {}".format(idx))


No entry for 0
don't
it's
i'm
can't
that's
you're
doesn't
didn't
he's
won't
isn't
let's
what's
obama's
i've
we're
aren't
wouldn't
i'll
they're
she's
i'd
you've
there's
wasn't
haven't
shouldn't
couldn't
you'll
stillsanders
trump's
youtu
fbid
we've
palestinei
here's
hasn't
we'll
people's
america's
gov't
women's
•
who's
you'd
president's
clinton's
god's
weren't
standwithrand
country's
they've
nation's
y'all
hillary's
they'll
children's
🇺🇸
he'll
today's
obummer
ain't
rinos
state's
where's
🇺🇸🇺🇸
doyourjob
huffingtonpost
bush's
american's
we'd
👍
washingtonpost
feelthebern
😍
lmao
woman's
party's
one's
government's
world's
killary
everyone's
❤️
he'd
paul's
they'd
fbcdn
man's
father's
mother's
else's
family's
nobillnobreak
administration's
hphotos
hadn't
bridenstine
it'll
randrally
hahaha
she'll
how's
isil
randpaul
cispa
bengazi
iran's
😊
bernie's
'the
sheeple
israel's
someone's
bernieorbust
😂
would've
veteran's
rino's
neverhillary
reid's
aspx
republican's
😡
ryan's
ya'll
thehill
1073741828
repub


In [21]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.15381079935942088

In [22]:
embedding_matrix.shape

(1113992, 100)

In [23]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           111399200 
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [27]:
try:
  time_start = time.time()

  history = model.fit(smaller_X_train, smaller_y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 4000000 samples, validate on 1000000 samples
Epoch 1/30
4000000/4000000 [==============================] - 105s 26us/sample - loss: 0.5202 - acc: 0.7528 - val_loss: 0.4607 - val_acc: 0.8283
Epoch 2/30
4000000/4000000 [==============================] - 104s 26us/sample - loss: 0.5028 - acc: 0.7611 - val_loss: 0.4750 - val_acc: 0.8209
Epoch 3/30
4000000/4000000 [==============================] - 103s 26us/sample - loss: 0.4973 - acc: 0.7638 - val_loss: 0.5064 - val_acc: 0.7976
Epoch 4/30
4000000/4000000 [==============================] - 105s 26us/sample - loss: 0.4939 - acc: 0.7654 - val_loss: 0.4652 - val_acc: 0.8242
Epoch 5/30
4000000/4000000 [==============================] - 102s 25us/sample - loss: 0.4913 - acc: 0.7667 - val_loss: 0.4651 - val_acc: 0.8251
Epoch 6/30
4000000/4000000 [==============================] - 102s 26us/sample - loss: 0.4894 - acc: 0.7675 - val_loss: 0.4627 - val_acc: 0.8283
Epoch 7/30
4000000/4000000 [==============================] - 104s 26us/samp

In [28]:
loss, accuracy = model.evaluate(smaller_X_train, smaller_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7755
Testing Accuracy:  0.8185


In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [35]:
wrong_preds.op_gender.value_counts()

W    302625
M    113489
Name: op_gender, dtype: int64

In [36]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.06777422721462319

In [37]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1927625,10977017,86569077,W,389345,Jane,You sounded like a fool.,Kirsten Gillibrand,Congress_Democratic,0.777178,M
2078802,11219782,77234050,W,398978,Kathy,"Wasn't finished with my post, I want the military to recognized that Fort Mccellan Alabama is a ...",Cathy McMorris Rodgers,Congress_Republican,0.728519,M
375560,3125651,23984618,W,82876,Carolyn,I feel better knowing you'll get nothing done in this committee based on your history of doing a...,Ann Wagner,Congress_Republican,0.740488,M
2153523,11294503,77234050,W,400424,Barb,So tell me again why we should bail them out!,Cathy McMorris Rodgers,Congress_Republican,0.826003,M
930243,5179297,46506154,M,161214,Marco,"Merry Christmas, Senator! To you and your beautiful family.",Marco Rubio,Congress_Republican,0.391529,W
1940138,10989530,86569077,W,389446,Joanne,If you want someone to lie to your face...who thinks it's okay to murder a baby in the womb gett...,Kirsten Gillibrand,Congress_Democratic,0.718247,M
1927800,10977192,86569077,W,389346,Jeanne,Thank you for representing me!,Kirsten Gillibrand,Congress_Democratic,0.626463,M
2126066,11267046,77234050,W,399739,Hank,This letter certainly makes it clear that the Republican House-caused government shutdown is all...,Cathy McMorris Rodgers,Congress_Republican,0.800879,M
1557113,7975407,99469231,M,296684,Dalia,WE LOVE YOU!,John McCain,Congress_Republican,0.217080,W
2087973,11228953,77234050,W,399295,Charlie,Obamacare is saving lives and money. Why are you so against this? There's room for improvement...,Cathy McMorris Rodgers,Congress_Republican,0.726627,M


In [38]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2001443,11050835,86569077,W,390046,Jim,I missed that one Jay....what Constitutional right does Rand Paul want repealed?,Kirsten Gillibrand,Congress_Democratic,0.999948,M
1912412,10961804,86569077,W,389338,Kristie,The President's job is to keep this country safe. Kirsten Gillibrand - Have you listened to the...,Kirsten Gillibrand,Congress_Democratic,0.999933,M
1908358,10957750,86569077,W,389335,Andrea,Thank you Sen Gillibrand for representing NY so well.,Kirsten Gillibrand,Congress_Democratic,0.999931,M
1932679,10982071,86569077,W,389363,Cheryl,"Senator Gillibrand of New York really has her priorities upside down. She, along with 91% of he...",Kirsten Gillibrand,Congress_Democratic,0.999885,M
1924180,10973572,86569077,W,389341,Ashley,"Thank you, Sen. Kirsten Gillibrand. I appreciated the response from your office.",Kirsten Gillibrand,Congress_Democratic,0.999881,M
2064389,11204143,48870736,W,394926,Michelle,Please hold this despicable man accountable for any laws broken. Thank you Representative McCollum.,Betty McCollum,Congress_Democratic,0.999880,M
373456,2653752,13876473,W,70137,Beth,Thank you Rep. Maloney - your wisdom in this matter is greatly appreciated byl those who appreci...,Carolyn Maloney,Congress_Democratic,0.999855,M
2043503,11092895,86569077,W,390561,Jennifer,I havent seen any Democrats saying this is not a good idea.. Rebublicans ( Rand Paul for exampl...,Kirsten Gillibrand,Congress_Democratic,0.999850,M
2026550,11075942,86569077,W,390298,Noel,Thank you Senator Kirsten Gillibrand.. Happy St. Patrick's Day,Kirsten Gillibrand,Congress_Democratic,0.999844,M
2001084,11050476,86569077,W,390043,Margaret,"The shooter took his mother's gun. If these guns were still banned, she wouldn't have it. Mr Ran...",Kirsten Gillibrand,Congress_Democratic,0.999817,M


In [39]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1667284,10169799,76267383,M,374064,Mary,God bless you Elizabeth! You are an inspiration!,Keith Rothfus,Congress_Republican,0.000031,W
887339,5136393,46506154,M,160982,Lora,Go Joni Ernst. Sending positive energy from NYC.,Marco Rubio,Congress_Republican,0.000144,W
181521,2358273,62246450,M,52502,Annie,Keep going Elizabeth! We support you!,Ron Wyden,Congress_Democratic,0.000169,W
1055857,5304911,46506154,M,161415,Marty,Elizabeth Warren is a REAL phony! Vote Scott Brown in!!!,Marco Rubio,Congress_Republican,0.000181,W
617835,4866889,46506154,M,160203,Genoveva,Ahora todo lo que le pase a ese viejo racista desgraciado es culpa de Obama! ... o sea para todo...,Marco Rubio,Congress_Republican,0.000187,W
2185085,11326065,95515852,M,402463,Paula,http://blogs.rollcall.com/moneyline/dingell-campaigns-funded-mostly-by-interest-groups/,Bill Huizenga,Congress_Republican,0.000205,W
517787,4766841,46506154,M,159905,Alice,I have supported you from the start. I met you in NH with Scott Brown . I Wanted you to be our ...,Marco Rubio,Congress_Republican,0.000206,W
1883166,10916094,10458808,M,385912,Stephen,But Takano does not support Kamala Harris! He just lost my vote!,Mark Takano,Congress_Democratic,0.000217,W
2166055,11307035,95515852,M,401117,Jennifer,"Happy to see Rep. Mia Love there also, enjoyed her speech at the Rally for Life in January.",Bill Huizenga,Congress_Republican,0.000234,W
1870185,10903113,10458808,M,385776,Hannah,I see Tulsi!,Mark Takano,Congress_Democratic,0.000274,W
